In [1]:
#export
import json
import sys
import os
import os.path
from pathlib import Path
from subprocess import call
import subprocess
import sys # system libraries, like arguments (argv)
import re # regular expressions
import pandas as pd
import glob
import os.path
import numpy as np
from pathlib import Path


import scipy.stats as stats
# import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols


from scipy import stats


import statsmodels
# print(statsmodels.__version__)  
# 0.8.0rc1

from statsmodels.sandbox.stats.multicomp import fdrcorrection0
# Pulled from https://pythonfordatascience.org/anova-python/



from timeit import default_timer as timer
import time



In [2]:
#export
def removeFile(filename):
    try:
        os.remove(filename)
    except OSError:
        pass   

In [3]:
#export
def makeFolders(folder_list):
    for directory in folder_list:
        if not directory: # Make sure not an empty string ""
            continue
        else:
            if not os.path.exists(directory):
                os.makedirs(directory)

In [4]:
# Anova test



# Loading data
df = pd.read_csv("https://raw.githubusercontent.com/Opensourcefordatascience/Data-sets/master/difficile.csv")
df.drop('person', axis= 1, inplace= True)

# Recoding value from numeric to string
df['dose'].replace({1: 'placebo', 2: 'low', 3: 'high'}, inplace= True)
    
# Gettin summary statistics
rp.summary_cont(df['libido'])

print(df.head())
print(df.dtypes)
print(df.shape)

results = ols('libido ~ C(dose)', data=df).fit()
results.summary()
print(results.summary())

NameError: name 'rp' is not defined

In [ ]:
class FilterANOVA():
  
    def __init__(self, 
         NF_out, 
         NF_path,
         col_data, 
         title,
         p_adj,
         meta_comparison="condition",
         searchlist = None,
         binRank = None,
         min_length = 100):
        
        
        self.NF_path = NF_path
        self.p_adj = p_adj
        self.searchlist = searchlist
        self.binRank = binRank
        self.min_length = min_length
        self.meta_comparison = meta_comparison
        
        self.title = title
        self.NF_out = NF_out
        self.wkdir = f'{self.NF_out}/unmapped/final'
        self.col_data = col_data
        self.df_col = pd.read_csv(self.col_data,sep="\t",names = ["sample","condition"])
        self.condition_list = self.df_col["condition"].unique() 
        
        
  
    def searchDF(self, df, HAS, rank, name):
        df = df.fillna("NA")
        if HAS:
            df = df.loc[df[rank]==name]
        else:
            df = df.loc[df[rank]!=name]
        return df
        
        
    def Filter(self, df):
        if self.searchlist == None or self.searchlist == "contig":
            return df
        else:
            for search in self.searchlist:           
                if len(df) > 0:   
                    HAS = search[0]
                    rank = search[1]
                    name = search[2]
                    
                    print(f"Searching with {HAS}:{rank}:{name}")
                    df = self.searchDF(df=df, HAS=HAS, rank=rank,name=name) 
            return df
        
        
     
    def runANOVA(self,df):
        print(f"Running Ttest with {len(df)}")
                

        
        
        
        

    def runSearch(self):
        files = glob.glob(f'{self.wkdir}/pileupCoverageNormalizedMatched/*.txt')
        meta_comparison = self.meta_comparison

        out = f'{self.NF_out}/FINAL_OUT/{self.title}'
        minout = f'{out}/minlength_{self.min_length}'
        makeFolders([f'{self.NF_out}/FINAL_OUT/',out])

        df = pd.read_csv(files[0],sep="\t",low_memory=False)
        df = self.Filter(df) 
        df = df[df[binRank] != "NA"]
        df = df[df[binRank].notna()]
        df_col = self.df_col
                

        
        samples = df_col["sample"].to_list()
        df_col = df_col.set_index("sample")
        meta_data = list(df_col)
        
        
        #Change dataframe to long
        df = df.T

        df_samples = df.loc[ samples , : ]
        df_binrank = df[df.index.str.match(self.binRank)]

        df = pd.concat((df_binrank,df_samples))        
        new_header = df.iloc[0] #grab the first row for the header
        df = df[1:] #take the data less the header row
        df.columns = new_header #set the header row as the df header
        contigs = df.columns
        num_contigs = len(list(contigs))
        df = pd.merge(df,df_col, left_on=df.index, right_on="sample")
        df = df.set_index("sample")
        
        
        #Loop through and run ANOVA for each contig
        
        
        x = 0
        contig = contigs[x]
        df2 = df[[contig]+[meta_comparison]].copy()
        df2[contig] = df2[contig].astype(float)
        print(df2.dtypes)      


        df2["temp"] = df2[contig]
        results = ols(f'temp ~ C({meta_comparison})', data=df2).fit()
#         print(results.summary()) 

        aov_table = sm.stats.anova_lm(results, typ=2)
        print(aov_table)
        
        
        
        
        
        
        
        
#         # With scipy
#         comparisons = list(df2[meta_comparison].unique())

#         lc = []
#         for comparison in comparisons:
#             temp_df = df2[df2[meta_comparison]==comparison].copy()
#             numbers = temp_df[contig]
#             lc.append(numbers)

#         fvalue, pvalue = stats.f_oneway(lc[0],lc[1],lc[2])
#         print(fvalue,pvalue)
        ########



        
#         print(df2.head(10))
        df.to_csv("test.txt",sep="\t")

        
        
#         dfs = []
#         for f in files:
#             print("---------------")
#             file = os.path.basename(f)
#             print("out:", out)
#             print(f' Working on file {file}')
#             df = pd.read_csv(f,sep="\t",low_memory=False)
#             print(f"Starting contigs: {len(df)}")

#             df = self.Filter(df)
#             print(df)

        
        
        
        

        

In [5]:
# NF = "/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/nextflow/RNAseq-Biome-Nextflow"
NF_path = "/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master"

NF_out = f"{NF_path}/NF_OUT" 
col_data = f"{NF_path}/sample_table.txt"
df_col = pd.read_csv(col_data,sep="\t",names=["sample","condition"])




searchlist = None
binRank = "contig"

title = "OUT_All_Contigs_bin_species_ANOVA"

F = FilterANOVA(NF_out=NF_out,
                NF_path = NF_path,
             col_data = col_data, 
             title=title, 
             searchlist=searchlist,
             meta_comparison = "condition",
             binRank = binRank,
             min_length = 100,
             p_adj = 0.05
            )

F.runSearch()
        
        


NameError: name 'FilterANOVA' is not defined

In [6]:
#export
class FilterTtest():
    
    def __init__(self, 
         NF_out, 
         NF_path,
         col_data, 
         title,
         control,
         treatment,
         p_adj,
         searchlist = None,
         binRank = None,
         min_length = 100):

        self.NF_path = NF_path
        self.p_adj = p_adj
        self.searchlist = searchlist
        self.binRank = binRank
        self.control = control
        self.treatment = treatment 
        self.min_length = min_length

        self.title = title
        self.NF_out = NF_out
        self.wkdir = f'{self.NF_out}/unmapped/final'
        self.col_data = col_data
        self.df_col = pd.read_csv(self.col_data,sep="\t",names = ["sample","condition"])
        self.condition_list = self.df_col["condition"].unique()

        
    def searchDF(self, df, HAS, rank, name):
        df = df.fillna("NA")
        if HAS:
            df = df.loc[df[rank]==name]
        else:
            
            df = df.loc[df[rank]!=name]
        return df
        
        
    def Filter(self, df):
        if self.searchlist == None:
            return df
        else:
            for search in self.searchlist:           
                if len(df) > 0:   
                    HAS = search[0]
                    rank = search[1]
                    name = search[2]
                    
                    print(f"Searching with {HAS}:{rank}:{name}")
                    df = self.searchDF(df=df, HAS=HAS, rank=rank,name=name) 
            return df

        

        
    def runTtest(self,df):
        print(f"Running Ttest with {len(df)}")
        
        # Get coverages by conditions 
        treatment_df = self.df_col[self.df_col["condition"]==self.treatment]
        treatment_list = list(treatment_df["sample"])
        control_df = self.df_col[self.df_col["condition"]==self.control]
        control_list = list(control_df["sample"])
        
        df_treatment = df[treatment_list]
        df_control = df[control_list]
        
        t_list = []
        p_val = []
        df["levine_statistic"] = np.nan
        df["levine_pval"] = np.nan
        df["t_statistic"] = np.nan
        df["t_pval"] = np.nan
#         df["t_padj"] = 0.0
        
        df = df.copy()
        for i in range(len(df)):
            treatment_vals = df_treatment.iloc[i,:]
            control_vals = df_control.iloc[i,:]
            
            lev = stats.levene(treatment_vals, control_vals)
            df["levine_statistic"].iloc[i] = lev[0]
            df["levine_pval"].iloc[i] = lev[1]
        
        
            ttest = stats.ttest_ind(treatment_vals, control_vals)
            df["t_statistic"].iloc[i] = ttest[0]
            df["t_pval"].iloc[i] = ttest[1]
        

        #Benjamini hochberg fdr correction
        df["t_padj"] = np.nan
        df_null = df[df["t_pval"].isnull()]
        df = df[~df["t_pval"].isnull()] # Get everything not null because statsmodel cant handle it

        padj = statsmodels.sandbox.stats.multicomp.fdrcorrection0(df["t_pval"], alpha=0.05, method='indep', is_sorted=False)        
        
        df["t_padj_reject"] = padj[0] 
        df["t_padj"] = list(padj[1])
        df = pd.concat([df,df_null], sort=False)

#         print(df)

    
        cols = list(df)
        
        statcols = ["contig", "name", "superkingdom", "kingdom", "phylum", "order",	"family", "genus", "species", "blast_pident", "blast_sseqid", "blast_evalue", "taxid", "blast_stitle", "filename", "length", "fasta"]
        statcols = ["levine_statistic","levine_pval","t_statistic","t_pval","t_padj","t_padj_reject"]
        newcols = [x for x in cols if x not in statcols]
        cols = statcols + newcols
        df = df[cols]

        
        return df
    
    def runSearch(self):
        files = glob.glob(f'{self.wkdir}/pileupCoverageNormalizedMatched/*.txt')

        out = f'{self.NF_out}/FINAL_OUT/{self.title}'
        ddout = f'{self.NF_out}/FINAL_OUT/{control}_{treatment}_doubledark'
        minout = f'{out}/minlength_{self.min_length}'
        makeFolders([f'{self.NF_out}/FINAL_OUT/',out, ddout])

        empties = []
        
                    
            #Run T test for no hits if it doesn't exist
        if os.path.exists(f'{ddout}/{title}_dark_all_FG_NoHits.txt') == False:
            df = pd.read_csv(f'{self.wkdir}/pileupCoverageNormalizedMatched/dark_all_FG_NoHits_pileupCoverageNormalizedMatched.tsv',sep="\t")
            df = self.runTtest(df)
            df_sig = df[df["t_padj_reject"]==True]
            f_sig = f'{ddout}/{control}_{treatment}_doubledark_double_dark_all_FG_NoHits_Significant.txt'
            f_df = f'{ddout}/{control}_{treatment}_doubledark_double_dark_all_FG_NoHits.txt'
            df_sig.to_csv(f_sig,sep="\t",index=None)
            df.to_csv(f_df,sep="\t",index=None)            


        
        
        dfs = []
        for f in files:
            print("---------------")
            file = os.path.basename(f)
            print("out:", out)
            print(f' Working on file {file}')
            df = pd.read_csv(f,sep="\t",low_memory=False)
            print(f"Starting contigs: {len(df)}")
            df = self.Filter(df)

            
            if len(df) > 0:
                if self.binRank != None:
                    df = df.groupby([self.binRank]).sum()
                    print(f"Binning by Rank {self.binRank}: Length: {len(df)}")

                
                
#                 print(df.head())
                df = self.runTtest(df)
#                 print(df.head())
                df_sig = df[df["t_padj_reject"]==True]
                f_sig = f'{out}/{self.title}_Significant_{file}'
                f_df = f'{out}/{self.title}_{file}'

                self.df_sig = df_sig
                if self.binRank != None:
                    df_sig.to_csv(f_sig,sep="\t")
                    df.to_csv(f_df,sep="\t")
                else:
                    df_sig.to_csv(f_sig,sep="\t",index=None)
                    df.to_csv(f_df,sep="\t",index=None)
                    
                
            else:
                
                empties.append(file)
                
#         if len(empties)  > 0:  
            
#             with open(f'{minout}/Empty_files.txt',"w") as outfile:
#                 for empty in empties:
#                     outfile.write(os.path.basename(empty)+"\n")            

 

In [7]:

NF_path = "/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master/"
NF_out = f"{NF_path}/NF_OUT" 
col_data = f"{NF_path}/sample_table.txt"


df_col = pd.read_csv(col_data,sep="\t",names=["sample","condition"])
items = list(df_col["condition"].unique())
conditions = [(items[i],items[j]) for i in range(len(items)) for j in range(i+1, len(items))]


permutation = conditions[0]
control = permutation[0]
treatment = permutation[1]



# searchlist = [(True,"phylum","NA")]
# searchlist = [(True,"superkingdom","NA")] # Everything without a superkingdom
# searchlist = [(True,"superkingdom","Bacteria")] # Everything without a superkingdom



control = "one_POLYA"
treatment = "TW4_POLYA"


searchlist = None


searchlist = [(True,"superkingdom","Bacteria")]

binRank = "species"

title = "Z_OUT_All_Bacteria_bin_species"

T = FilterTtest(NF_out=NF_out,
                NF_path = NF_path,
             col_data = col_data, 
             title=title, 
             searchlist=searchlist,
             control = control, 
             treatment = treatment, 
             binRank = binRank,
             min_length = 100,
             p_adj = 0.05
            )

T.runSearch()



Running Ttest with 6
---------------
out: /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master//NF_OUT/FINAL_OUT/Z_OUT_All_Bacteria_bin_species
 Working on file dark_all_FG_pileupCoverageNormalizedMatched.txt
Starting contigs: 2
Searching with True:superkingdom:Bacteria
Binning by Rank species: Length: 2
Running Ttest with 2
---------------
out: /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master//NF_OUT/FINAL_OUT/Z_OUT_All_Bacteria_bin_species
 Working on file all_pileupCoverageNormalizedMatched.txt
Starting contigs: 1199
Searching with True:superkingdom:Bacteria
Binning by Rank species: Length: 7
Running Ttest with 7


/Users/m/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2352: RuntimeWarning: invalid value encountered in double_scalars
  W = numer / denom
/Users/m/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
df = pd.DataFrame({"name": ['Alfred', 'Batman', np.nan],

                   "toy": [np.nan, 'Batmobile', 'Bullwhip']})


df

df_nan = df[~df["name"].isnull()]
df_nan

,name,toy
0,Alfred,NaN
1,Batman,Batmobile


In [9]:

# # Ranks
ranks = ['superkingdom',
 'kingdom',
 'phylum',
 'order',
 'family',
 'genus',
 'species',"name"]


superkingdoms = ['Bacteria', 'Viruses', 'Eukaryota', "NA", 'Archaea']

In [10]:
for superkingdom in superkingdoms:
    print(f"**** working on {superkingdom}")
    searchlist = [(True,"superkingdom",superkingdom)]
    for binRank in ranks:
  
        makeFolders(f"OUT_{superkingdom}")
        title = f"A_B_{superkingdom}_Bin_{binRank}"
        
        T = FilterTtest(NF_out=NF_out,
                        NF_path = NF_path,
                     col_data = col_data, 
                     title=title, 
                     searchlist=searchlist,
                     control = control, 
                     treatment = treatment, 
                     binRank = binRank,
                     min_length = 100,
                     p_adj = 0.05
                    )

        T.runSearch()


**** working on Bacteria
Running Ttest with 6
---------------
out: /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master//NF_OUT/FINAL_OUT/A_B_Bacteria_Bin_superkingdom
 Working on file dark_all_FG_pileupCoverageNormalizedMatched.txt
Starting contigs: 2
Searching with True:superkingdom:Bacteria
Binning by Rank superkingdom: Length: 1
Running Ttest with 1
---------------
out: /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master//NF_OUT/FINAL_OUT/A_B_Bacteria_Bin_superkingdom
 Working on file all_pileupCoverageNormalizedMatched.txt
Starting contigs: 1199
Searching with True:superkingdom:Bacteria
Binning by Rank superkingdom: Length: 1
Running Ttest with 1
Running Ttest with 6
---------------
out: /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master//NF_OUT/FINAL_OUT/A_B_Bacteria_Bin_kingdom
 Working on file dark_all_FG_pileupCoverageNormalizedMatc

Starting contigs: 1199
Searching with True:superkingdom:Viruses
Binning by Rank genus: Length: 1
Running Ttest with 1
Running Ttest with 6
---------------
out: /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master//NF_OUT/FINAL_OUT/A_B_Viruses_Bin_species
 Working on file dark_all_FG_pileupCoverageNormalizedMatched.txt
Starting contigs: 2
Searching with True:superkingdom:Viruses
---------------
out: /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master//NF_OUT/FINAL_OUT/A_B_Viruses_Bin_species
 Working on file all_pileupCoverageNormalizedMatched.txt
Starting contigs: 1199
Searching with True:superkingdom:Viruses
Binning by Rank species: Length: 1
Running Ttest with 1
Running Ttest with 6
---------------
out: /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master//NF_OUT/FINAL_OUT/A_B_Viruses_Bin_name
 Working on file dark_all_FG_pileupCoverageNorma

Running Ttest with 6
---------------
out: /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master//NF_OUT/FINAL_OUT/A_B_NA_Bin_genus
 Working on file dark_all_FG_pileupCoverageNormalizedMatched.txt
Starting contigs: 2
Searching with True:superkingdom:NA
---------------
out: /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master//NF_OUT/FINAL_OUT/A_B_NA_Bin_genus
 Working on file all_pileupCoverageNormalizedMatched.txt
Starting contigs: 1199
Searching with True:superkingdom:NA
Running Ttest with 6
---------------
out: /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master//NF_OUT/FINAL_OUT/A_B_NA_Bin_species
 Working on file dark_all_FG_pileupCoverageNormalizedMatched.txt
Starting contigs: 2
Searching with True:superkingdom:NA
---------------
out: /Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/chlamydia/RNAseq-Biome-master//NF_O

In [11]:
#export
import argparse
def parse_arguments():
        parser = argparse.ArgumentParser(description='filter quantify and graph')
        parser.add_argument('--Nextflow_Out', action= 'store', metavar='--Nextflow_Out') 
        parser.add_argument('--Nextflow_path', action= 'store', metavar='--Nextflow_path') 
        args = parser.parse_args()
        return args


In [12]:
#export
if __name__=="__main__":
    args = parse_arguments()

    NF_out = args.Nextflow_Out
    NF_path = args.Nextflow_path
    col_data = f"{NF_path}/sample_table.txt"

    
    levels = ["all","dark_all_FG"]

    ranks = ['superkingdom',
 'kingdom',
 'phylum',
 'order',
 'family',
 'genus',
 'species',"name"]


    superkingdoms = ['Bacteria', 'Viruses', 'Eukaryota', 'NA', 'Archaea']    
    
    
    # Make permutation of conditions
    col_data = f"{NF_path}/sample_table.txt"
    df_col = pd.read_csv(col_data,sep="\t",names=["sample","condition"])
    items = list(df_col["condition"].unique())
    conditions = [(items[i],items[j]) for i in range(len(items)) for j in range(i+1, len(items))]
    

    


    # This will run all contigs 
    searchlist = None
    binRank = None

    
    
    for permutation in conditions:
        control = permutation[0]
        treatment = permutation[1]

        title = f"{control}_{treatment}_OUT_ALL_CONTIGS"

        T = FilterTtest(NF_out=NF_out, 
                     NF_path=NF_path,
                     col_data = col_data, 
                     title=title, 
                     searchlist=searchlist,
                     control = control, 
                     treatment = treatment, 
                     binRank = binRank,
                     min_length = 100,
                     p_adj = 0.05
                    )

        T.runSearch()  
        


    for permutation in conditions:
        control = permutation[0]
        treatment = permutation[1]

        for superkingdom in superkingdoms:
            print(f"**** working on {superkingdom}")
            searchlist = [(True,"superkingdom",superkingdom)]
            for binRank in ranks:

                makeFolders(f"OUT_{superkingdom}")
                title = f"{control}_{treatment}_{superkingdom}_Bin_{binRank}"

                T = FilterTtest(NF_out=NF_out, 
                             NF_path=NF_path,
                             col_data = col_data, 
                             title=title, 
                             searchlist=searchlist,
                             control = control, 
                             treatment = treatment, 
                             binRank = binRank,
                             min_length = 100,
                             p_adj = 0.05
                            )

                T.runSearch()
      

usage: ipykernel_launcher.py [-h] [--Nextflow_Out --Nextflow_Out]
                             [--Nextflow_path --Nextflow_path]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/m/Library/Jupyter/runtime/kernel-1d46cadb-aa23-4d58-8648-0ed0bcaf6e06.json


SystemExit: 2

/Users/m/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
import fire
!python notebook2script.py 5FilterTtest.ipynb

Converted 5FilterTtest.ipynb to nb_5FilterTtest.py
